<a href="https://colab.research.google.com/github/greenmetro/FarmHelper/blob/master/PMDA_%EB%AC%B8%EC%84%9C_%EA%B8%B0%EB%B0%98_Gemma_%EB%AA%A8%EB%8D%B8_%ED%95%99%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ==============================================================================
# Step 1: 환경 설정 및 라이브러리 설치
# ==============================================================================
# GPU 가속을 위해 필요한 라이브러리들을 설치합니다.
# 'pdfplumber'는 표/이미지 추출을 위해, 'transformers'는 LLaVA 모델을 위해, 'Pillow'는 이미지 처리를 위해 필요합니다.
!pip uninstall -y Pillow torchvision
!pip install transformers accelerate peft bitsandbytes pypdf python-dotenv datasets pdfplumber markdownify Pillow torchvision trl --no-cache-dir

# 필요한 라이브러리를 불러옵니다.
import requests
import json
import os
from pathlib import Path
import pdfplumber
import markdownify
import torch
from datasets import Dataset
from transformers import AutoProcessor, LlavaForConditionalGeneration, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer
from PIL import Image
import logging

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# ==============================================================================
# Step 2: PMDA 문서 다운로드 및 텍스트/표/이미지 추출
# ==============================================================================
def download_and_parse_documents(urls, save_dir="pmda_docs", img_dir="pmda_images"):
    """
    PMDA 문서 URL 리스트에서 PDF 파일을 다운로드하고, 텍스트, 표, 이미지를 추출합니다.

    Args:
        urls (list): PDF 파일 URL 리스트.
        save_dir (str): PDF 파일을 저장할 디렉토리.
        img_dir (str): 추출된 이미지를 저장할 디렉토리.

    Returns:
        dict: 파일 경로, 추출된 텍스트, 그리고 이미지 파일 경로를 담은 딕셔너리.
    """
    Path(save_dir).mkdir(exist_ok=True)
    Path(img_dir).mkdir(exist_ok=True)
    extracted_data = {}

    for url in urls:
        file_name = url.split('/')[-1]
        file_path = os.path.join(save_dir, file_name)

        try:
            logger.info(f"Downloading {file_name}...")
            response = requests.get(url, timeout=30)
            response.raise_for_status()

            with open(file_path, 'wb') as f:
                f.write(response.content)

            logger.info(f"Extracting text, tables, and images from {file_name}...")
            document_text = ""
            extracted_images = []

            with pdfplumber.open(file_path) as pdf:
                for i, page in enumerate(pdf.pages):
                    # 페이지 내의 모든 텍스트 추출
                    document_text += page.extract_text() or ""

                    # 페이지 내의 모든 표 추출 및 마크다운으로 변환
                    tables = page.extract_tables()
                    for table_data in tables:
                        if not table_data: continue
                        header = table_data[0]
                        rows = table_data[1:]

                        table_string = "\n| " + " | ".join(header) + " |\n"
                        table_string += "| " + " | ".join(['---'] * len(header)) + " |\n"
                        for row in rows:
                            table_string += "| " + " | ".join([str(cell) if cell is not None else '' for cell in row]) + " |\n"

                        document_text += "\n" + table_string + "\n"

                    # 페이지 내의 이미지 추출 및 저장
                    images = page.images
                    for img_data in images:
                        img_name = f"{file_name}_page_{i+1}_{img_data['x']:g}_{img_data['y']:g}.png"
                        img_path = os.path.join(img_dir, img_name)

                        if 'stream' in img_data:
                            img = Image.open(img_data['stream'])
                            img.save(img_path)
                            extracted_images.append(img_path)

            # 불필요한 공백과 개행 제거
            clean_text = ' '.join(document_text.split())
            extracted_data[file_name] = {
                "text": clean_text,
                "images": extracted_images
            }

        except requests.exceptions.RequestException as e:
            logger.error(f"Failed to download {url}: {e}")
        except Exception as e:
            logger.error(f"Failed to extract data from {file_name}: {e}")

    return extracted_data

# PMDA 문서의 예시 URL 리스트
pmda_urls = [
    "https://www.pmda.go.jp/drugs/2024/P20240219001/300242000_30200AMX00502_B100_1.pdf"
]
downloaded_docs = download_and_parse_documents(pmda_urls)

# ==============================================================================
# Step 3: 학습용 데이터셋 생성 및 변환
# ==============================================================================
def create_multimodal_instruction_dataset(extracted_data):
    """
    추출된 텍스트, 표, 이미지를 Gemma 모델 학습에 적합한 JSONL 형식으로 변환합니다.
    """
    dataset_list = []

    for filename, data in extracted_data.items():
        document_part = data["text"][:2000] # 앞부분 2000자 사용
        image_path = data["images"][0] if data["images"] else None # 첫 번째 이미지 사용

        if image_path:
            # 멀티모달 학습용 프롬프트 구성
            instruction = f"문서와 이미지를 분석하고 다음 질문에 답해줘: 이 보고서의 핵심 내용과 이미지에 나타난 그래프의 의미는 무엇인가요?"
            output = "이 문서는 의약품의 임상 데이터를 다루며, 첨부된 그래프는 약물 투여량에 따른 환자의 반응률을 보여줍니다. 그래프는 용량 증가에 따라 반응률이 높아지는 경향을 나타냅니다."

            dataset_list.append({
                "instruction": instruction,
                "image_path": image_path,
                "output": output
            })
        else:
            # 이미지가 없는 경우 텍스트만 사용
            instruction = f"다음 문서를 요약해줘: {document_part}"
            output = "이 문서는 의약품 신청서에 대한 기술적 평가 보고서입니다."

            dataset_list.append({
                "instruction": instruction,
                "output": output
            })

    return Dataset.from_list(dataset_list)

train_dataset = create_multimodal_instruction_dataset(downloaded_docs)
logger.info("Generated training dataset:")
logger.info(train_dataset)
logger.info("First example in the dataset:")
if len(train_dataset) > 0:
    logger.info(train_dataset[0])
else:
    logger.warning("Training dataset is empty. Cannot display the first example.")


# ==============================================================================
# Step 4: LLaVA 모델 추가 학습 (Fine-tuning)
# ==============================================================================
def load_and_train_llava(dataset):
    """
    LLaVA 모델을 로드하고 LoRA 기법을 사용하여 추가 학습을 진행합니다.
    """
    if len(dataset) == 0:
        logger.error("Cannot train the model with an empty dataset.")
        return

    model_name = "llava-hf/llava-1.5-7b-hf"
    new_model_name = "llava-1.5-7b-finetuned-pmda"

    # 모델을 4-bit 양자화로 로드하기 위한 설정
    bnb_config = BitsandBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
    )

    # 프로세서와 모델 로드 (LLaVA는 별도의 프로세서가 필요)
    logger.info("Loading processor and model...")
    processor = AutoProcessor.from_pretrained(model_name)
    model = LlavaForConditionalGeneration.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto"
    )

    # LoRA 설정 (PEFT)
    peft_config = LoraConfig(
        r=16,
        lora_alpha=32,
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
    )
    model = get_peft_model(model, peft_config)
    model.print_trainable_parameters()

    # 데이터셋 전처리 함수
    def preprocess_multimodal(examples):
        image_path = examples['image_path']
        image = Image.open(image_path).convert("RGB")
        prompt = examples['instruction']

        # LLaVA용 프롬프트 형식
        full_prompt = f"USER: <image>\n{prompt} ASSISTANT: {examples['output']}"

        # 모델 입력에 맞게 토큰화 및 이미지 전처리
        inputs = processor(text=full_prompt, images=image, return_tensors="pt")

        return {
            "input_ids": inputs.input_ids[0],
            "attention_mask": inputs.attention_mask[0],
            "pixel_values": inputs.pixel_values[0],
            "labels": inputs.input_ids[0] # self-supervised learning
        }

    # 훈련 매개변수 설정
    training_args = TrainingArguments(
        output_dir="./results",
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        learning_rate=2e-4,
        logging_steps=10,
        max_steps=100,
        fp16=True,
        optim="paged_adamw_8bit",
        save_strategy="epoch"
    )

    # SFTTrainer를 사용한 모델 학습
    # LLaVA는 데이터셋을 수동으로 전처리해야 하므로, SFTTrainer를 직접 사용하기보다
    # `transformers.Trainer`를 사용하는 것이 더 적합합니다.
    # 여기서는 SFTTrainer의 데이터 로딩 기능을 활용하고, 실제 학습은 Trainer를 흉내냅니다.
    logger.warning("SFTTrainer is used for simplicity, but a custom Trainer might be better for complex multimodal tasks.")

    trainer = SFTTrainer(
        model=model,
        train_dataset=dataset,
        peft_config=peft_config,
        dataset_text_field="instruction",
        max_seq_length=2048,
        tokenizer=processor.tokenizer,
        args=training_args,
    )

    # LLaVA는 별도의 이미지 전처리 로직이 필요하므로 직접 학습 루프를 구현하거나
    # `transformers.Trainer`를 사용하는 것이 더 정확합니다.
    # 아래 code는 LLaVA의 작동 방식을 설명하기 위한 단순화된 예시입니다.
    logger.info("Starting model training...")
    trainer.train()

    # 학습된 모델 저장
    trainer.save_model(new_model_name)
    logger.info(f"Model saved to ./{new_model_name}")

# 메인 실행
load_and_train_llava(train_dataset)

Found existing installation: pillow 12.0.0
Uninstalling pillow-12.0.0:
  Successfully uninstalled pillow-12.0.0
Found existing installation: torchvision 0.24.0
Uninstalling torchvision-0.24.0:
  Successfully uninstalled torchvision-0.24.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 162.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 5.49.1 requires pillow<12.0,>=8.0, but you have pillow 12.0.0 which is incompatible.


ERROR:__main__:Failed to extract data from 300242000_30200AMX00502_B100_1.pdf: 'x'
ERROR:__main__:Cannot train the model with an empty dataset.
